In [2]:
import scanpy as sc
import numpy as np

In [5]:
fly_ad = sc.read("export_data/fly_cell_atlas_uce_adata_33layer.h5ad", cache=True)
fly_ad

/lfs/ampere5/0/yanay/env/micromamba/envs/dogma/lib/python3.11/site-packages/anndata/_core/anndata.py:1825: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


AnnData object with n_obs × n_vars = 506660 × 13827
    obs: 'consensus_community_annotation', 'Barcode', 'age', 'batch_id', 'sex', 'tissue', 'harmonised_annotation', 'broad_annotation', 'n_genes'
    var: 'ID-0', 'n_cells'
    uns: 'neighbors', 'umap'
    obsm: 'X_uce', 'X_umap'
    obsp: 'connectivities', 'distances'

In [6]:
vcs = fly_ad.obs["broad_annotation"].value_counts()

common_cell_types =  list((vcs[vcs >= 100]).index)[1:]
common_cell_types

['epithelial cell',
 'muscle system',
 'sensory organ cell',
 'neuron',
 'Skeletor positive cells',
 'female reproductive system',
 'fat body',
 'tracheal cell',
 'germline cell',
 'excretory system',
 'hemocyte',
 'oenocyte',
 'glial cell',
 'gut cell',
 'male reproductive system',
 'artefact',
 'fat cell',
 'pigment cell',
 'gland',
 'somatic precursor cell',
 'mechanosensory neuron  of haltere',
 'cardial cell',
 'post-mitotic nurse cells; endocycling ']

In [7]:
fly_ad_common = fly_ad[(fly_ad.obs["broad_annotation"].isin(common_cell_types)) & (~fly_ad.obs["broad_annotation"].isin(["unknown", "artefact"]))].copy()
display(fly_ad_common)
fly_ad_common.obs["broad_annotation"].value_counts()

/lfs/ampere5/0/yanay/env/micromamba/envs/dogma/lib/python3.11/site-packages/anndata/_core/anndata.py:1825: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


AnnData object with n_obs × n_vars = 307241 × 13827
    obs: 'consensus_community_annotation', 'Barcode', 'age', 'batch_id', 'sex', 'tissue', 'harmonised_annotation', 'broad_annotation', 'n_genes'
    var: 'ID-0', 'n_cells'
    uns: 'neighbors', 'umap'
    obsm: 'X_uce', 'X_umap'
    obsp: 'connectivities', 'distances'

broad_annotation
epithelial cell                           68058
muscle system                             52004
sensory organ cell                        29543
neuron                                    28568
Skeletor positive cells                   20793
female reproductive system                19068
fat body                                  17125
tracheal cell                             10794
germline cell                             10071
excretory system                           9386
hemocyte                                   7899
oenocyte                                   7588
glial cell                                 7080
gut cell                                   6629
male reproductive system                   4454
fat cell                                   2488
pigment cell                               2014
gland                                      1462
somatic precursor cell                      795
mechanosensory neuron  of haltere           678
cardial cell           

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score


In [9]:
X = fly_ad_common.obsm["X_uce"]
y = fly_ad_common.obs["broad_annotation"].values

In [10]:
# Note: this for loop was written by Chat GPT version 5.2 Thinking

kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

accs = []


for fold, (train_idx, val_idx) in enumerate(kf.split(X, y), start=1):
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    model = LogisticRegression(n_jobs=-1, max_iter=200)
    
    model.fit(X_train, y_train)

    y_pred = model.predict(X_val)

    acc = accuracy_score(y_val, y_pred)
    accs.append(acc)

    print(f"Fold {fold}: acc={acc:.4f}")

print(f"\nMean over folds: acc={np.mean(accs):.4f} ± {np.std(accs):.4f}")

/lfs/ampere5/0/yanay/env/micromamba/envs/dogma/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)
/lfs/ampere5/0/yanay/env/micromamba/envs/dogma/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:406: ConvergenceWarning: lbfgs failed to converge after 200 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=200).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold 1: acc=0.8058


/lfs/ampere5/0/yanay/env/micromamba/envs/dogma/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)
/lfs/ampere5/0/yanay/env/micromamba/envs/dogma/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:406: ConvergenceWarning: lbfgs failed to converge after 200 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=200).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold 2: acc=0.8091


/lfs/ampere5/0/yanay/env/micromamba/envs/dogma/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)
/lfs/ampere5/0/yanay/env/micromamba/envs/dogma/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:406: ConvergenceWarning: lbfgs failed to converge after 200 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=200).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...

/lfs/ampere5/0/yanay/env/micromamba/envs/dogma/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)
/lfs/ampere5/0/yanay/env/micromamba/envs/dogma/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:406: ConvergenceWarning: lbfgs failed to converge after 200 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=200).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold 9: acc=0.8061


/lfs/ampere5/0/yanay/env/micromamba/envs/dogma/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


Fold 10: acc=0.8083

Mean over folds: acc=0.8060 ± 0.0017


/lfs/ampere5/0/yanay/env/micromamba/envs/dogma/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:406: ConvergenceWarning: lbfgs failed to converge after 200 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=200).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Check scores Random

In [11]:
X = fly_ad_common.obsm["X_uce"]
y = fly_ad_common.obs["broad_annotation"].values

In [12]:
# Note: this for loop was written by Chat GPT version 5.2 Thinking

kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

accs = []


for fold, (train_idx, val_idx) in enumerate(kf.split(X, y), start=1):
    np.random.shuffle(y)
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    model = LogisticRegression(n_jobs=-1, max_iter=200)
    
    model.fit(X_train, y_train)

    y_pred = model.predict(X_val)

    acc = accuracy_score(y_val, y_pred)
    accs.append(acc)

    print(f"Fold {fold}: acc={acc:.4f}")

print(f"\nMean over folds: acc={np.mean(accs):.4f} ± {np.std(accs):.4f}")

/tmp/user/21290/ipykernel_796686/398829837.py:9: UserWarning: you are shuffling a 'Categorical' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(y)
/lfs/ampere5/0/yanay/env/micromamba/envs/dogma/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)
/lfs/ampere5/0/yanay/env/micromamba/envs/dogma/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:406: ConvergenceWarning: lbfgs failed to converge after 200 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=200).
You might also want to scale the data as shown in:
    https://scikit-learn.org

Fold 1: acc=0.2195


/lfs/ampere5/0/yanay/env/micromamba/envs/dogma/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)
/lfs/ampere5/0/yanay/env/micromamba/envs/dogma/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:406: ConvergenceWarning: lbfgs failed to converge after 200 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=200).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/tmp/user/21290/ipykernel_796686/398829837.py:9: UserWarning: you are shuffling a 'Cat

Fold 2: acc=0.2196


/lfs/ampere5/0/yanay/env/micromamba/envs/dogma/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)
/lfs/ampere5/0/yanay/env/micromamba/envs/dogma/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:406: ConvergenceWarning: lbfgs failed to converge after 200 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=200).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/tmp/user/21290/ipykernel_796686/398829837.py:9: UserWarning: you are shuffling a 'Cat

...

/lfs/ampere5/0/yanay/env/micromamba/envs/dogma/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


Fold 9: acc=0.2185


/tmp/user/21290/ipykernel_796686/398829837.py:9: UserWarning: you are shuffling a 'Categorical' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(y)
/lfs/ampere5/0/yanay/env/micromamba/envs/dogma/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


Fold 10: acc=0.2200

Mean over folds: acc=0.2211 ± 0.0019


/lfs/ampere5/0/yanay/env/micromamba/envs/dogma/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:406: ConvergenceWarning: lbfgs failed to converge after 200 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=200).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
